## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Tilichiki,RU,60.4667,166.1000,52.772,74,100,2.16,overcast clouds
1,1,Ushuaia,AR,-54.8000,-68.3000,39.092,60,20,4.12,few clouds
2,2,Hermanus,ZA,-34.4187,19.2345,51.080,88,99,3.27,overcast clouds
3,3,Punta Arenas,CL,-53.1500,-70.9167,39.488,70,75,6.17,broken clouds
4,4,Grindavik,IS,63.8424,-22.4338,54.338,100,100,9.26,moderate rain


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp=float(input("What is the minimum temperature you would like for your trip? "))
max_temp=float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
7,7,Amapa,BR,1.0000,-52.0000,80.924,93,76,0.79,broken clouds
13,13,The Valley,AI,18.2170,-63.0578,87.926,66,40,7.72,scattered clouds
17,17,Takoradi,GH,4.8845,-1.7554,75.488,88,100,4.86,overcast clouds
18,18,Pleasanton,US,37.6624,-121.8747,87.890,44,0,3.09,clear sky
22,22,Butaritari,KI,3.0707,172.7902,81.986,75,12,8.28,few clouds
...,...,...,...,...,...,...,...,...,...,...
672,672,Garden City,US,42.3256,-83.3310,84.920,43,40,3.09,scattered clouds
674,674,Tinaquillo,VE,9.9186,-68.3047,78.584,86,91,1.30,overcast clouds
680,680,Dwarka,IN,22.2394,68.9678,82.508,83,100,7.28,overcast clouds
685,685,Darnah,LY,32.7670,22.6367,80.006,75,0,7.06,clear sky


In [14]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# No null files, so we are good

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
7,Amapa,BR,80.924,broken clouds,1.0000,-52.0000,
13,The Valley,AI,87.926,scattered clouds,18.2170,-63.0578,
17,Takoradi,GH,75.488,overcast clouds,4.8845,-1.7554,
18,Pleasanton,US,87.890,clear sky,37.6624,-121.8747,
22,Butaritari,KI,81.986,few clouds,3.0707,172.7902,
24,Arraial Do Cabo,BR,75.434,few clouds,-22.9661,-42.0278,
26,Hami,CN,81.068,clear sky,42.8000,93.4500,
28,Aguimes,ES,81.932,clear sky,27.9054,-15.4461,
29,Jamestown,US,78.800,scattered clouds,42.0970,-79.2353,
30,Saint Anthony,US,80.366,overcast clouds,45.0205,-93.2180,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"]=f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [11]:
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
7,Amapa,BR,80.924,broken clouds,1.0000,-52.0000,
13,The Valley,AI,87.926,scattered clouds,18.2170,-63.0578,CeBlue Villas & Beach Resort
17,Takoradi,GH,75.488,overcast clouds,4.8845,-1.7554,Raybow International Hotel
18,Pleasanton,US,87.890,clear sky,37.6624,-121.8747,Four Points by Sheraton Pleasanton
22,Butaritari,KI,81.986,few clouds,3.0707,172.7902,Isles Sunset Lodge
...,...,...,...,...,...,...,...
672,Garden City,US,84.920,scattered clouds,42.3256,-83.3310,Big Trees Mobile Home Community
674,Tinaquillo,VE,78.584,overcast clouds,9.9186,-68.3047,Hotel Los Copihues
680,Dwarka,IN,82.508,overcast clouds,22.2394,68.9678,Hotel Narayan Avezika Comfort
685,Darnah,LY,80.006,clear sky,32.7670,22.6367,فندق الفردوس


In [19]:
# 7. Drop the rows where there is no Hotel Name.
import numpy as np
hotel_df['Hotel Name'].replace('',np.nan,inplace=True)
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
7,Amapa,BR,80.924,broken clouds,1.0000,-52.0000,NaN
13,The Valley,AI,87.926,scattered clouds,18.2170,-63.0578,CeBlue Villas & Beach Resort
17,Takoradi,GH,75.488,overcast clouds,4.8845,-1.7554,Raybow International Hotel
18,Pleasanton,US,87.890,clear sky,37.6624,-121.8747,Four Points by Sheraton Pleasanton
22,Butaritari,KI,81.986,few clouds,3.0707,172.7902,Isles Sunset Lodge
...,...,...,...,...,...,...,...
672,Garden City,US,84.920,scattered clouds,42.3256,-83.3310,Big Trees Mobile Home Community
674,Tinaquillo,VE,78.584,overcast clouds,9.9186,-68.3047,Hotel Los Copihues
680,Dwarka,IN,82.508,overcast clouds,22.2394,68.9678,Hotel Narayan Avezika Comfort
685,Darnah,LY,80.006,clear sky,32.7670,22.6367,فندق الفردوس


In [20]:
clean_hotel_df=hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
13,The Valley,AI,87.926,scattered clouds,18.2170,-63.0578,CeBlue Villas & Beach Resort
17,Takoradi,GH,75.488,overcast clouds,4.8845,-1.7554,Raybow International Hotel
18,Pleasanton,US,87.890,clear sky,37.6624,-121.8747,Four Points by Sheraton Pleasanton
22,Butaritari,KI,81.986,few clouds,3.0707,172.7902,Isles Sunset Lodge
24,Arraial Do Cabo,BR,75.434,few clouds,-22.9661,-42.0278,Pousada Porto Praia
...,...,...,...,...,...,...,...
672,Garden City,US,84.920,scattered clouds,42.3256,-83.3310,Big Trees Mobile Home Community
674,Tinaquillo,VE,78.584,overcast clouds,9.9186,-68.3047,Hotel Los Copihues
680,Dwarka,IN,82.508,overcast clouds,22.2394,68.9678,Hotel Narayan Avezika Comfort
685,Darnah,LY,80.006,clear sky,32.7670,22.6367,فندق الفردوس


In [21]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [23]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<d1>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [24]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))